# 1. Leibnitz's Formula for PI

In [ ]:
from __future__ import division
import math

def estimate_one_fourth_pi(number_of_terms):
    indices = range(number_of_terms)
    sum = 0
    for i in indices:
        term = ((-1) ** i) * (1 / (2 * i + 1))
        sum += term
    return sum


In [5]:
math.fabs(estimate_one_fourth_pi(10) - math.pi / 4)

0.024938258665097468

In [6]:
math.fabs(estimate_one_fourth_pi(100) - math.pi / 4)

0.0024999375078098574

In [7]:
math.fabs(estimate_one_fourth_pi(1000) - math.pi / 4)

0.00024999993749974525

# 2. Sorting Algorithms

## Bubble Sort

In [9]:
import numpy.random as npr
from timeit import default_timer as timer

def bubble_sort(numerical_list, return_loops=False):
    loops = 0
    while True:
        swap_made = False
        loops += 1    
        for i in range(len(numerical_list) - 1):
            if numerical_list[i] > numerical_list[i + 1]:
                # Slick tuple swap
                (numerical_list[i], numerical_list[i + 1]) = (numerical_list[i + 1], numerical_list[i])
                swap_made = True
        if not swap_made:
            if return_loops:
                return (numerical_list, loops)
            else:
                return (numerical_list)

In [10]:
bubble_sort([5, 0, 9, 8, 7, 1, 4, 3, 2, 6], return_loops=True)

([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 7)

## Comb Sort

In [12]:
def comb_sort(numerical_list, scaling_factor, return_loops=False):
    loops = 0
    gap = len(numerical_list)
    while True:
        swap_made = False
        gap //= scaling_factor
        gap = int(gap)
        loops += 1    
        for i in range(len(numerical_list) - gap):
            if numerical_list[i] > numerical_list[i + gap]:
                # Slick tuple swap pt. 2
                (numerical_list[i], numerical_list[i + gap]) = (numerical_list[i + gap], numerical_list[i])
                swap_made = True
        if not swap_made:
            if return_loops:
                return (numerical_list, loops)
            else:
                return (numerical_list)

In [13]:
comb_sort([5, 0, 9, 8, 7, 1, 4, 3, 2, 6], scaling_factor=1.3, return_loops=True)

([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 6)

## Performance Analysis

In [14]:
bubble_sum = 0
comb_sum = 0

for i in range(100000):
    numbers = [npr.randint(0, 1000) for x in xrange(10)]
    
    bubble_start = timer()
    bubble_sort(numbers)
    bubble_end = timer()
    bubble_sum += (bubble_end - bubble_start)

    comb_start = timer()
    comb_sort(numbers, 1.5)
    comb_end = timer()
    comb_sum += (comb_end - comb_start)

In [16]:
"Bubble avg: %f" % (bubble_sum / 100000)

'Bubble avg: 0.000027'

In [17]:
"Comb avg: %f" % (comb_sum / 100000)

'Comb avg: 0.000004'

In [18]:
"Comb is %f times quicker than bubble" % (bubble_sum / comb_sum)

'Comb is 7.298699 times quicker than bubble'

# 3. Decimal and Sexagesimal Numbers

## Useful Functions

In [20]:
def has_decimal(number):
    return "." in str(number)

In [21]:
def get_error(number):
        chars = str(number)
        is_int = not has_decimal(number)

        counter = 0
        error = 0
        if is_int:
            for char in reversed(chars):
                if char != "0":
                    break
                counter += 1
            error = 10 ** counter
        else:
            counter = len(str(number - int(number))[2:])
            error = 10 ** (-counter)

        return error

In [22]:
def get_places(number):
    chars = str(number)

    # experimental hack to deal with scientific notation prblem
    if "e" in chars:
        return int(chars[-2:]) - 1
    
    counter = 0
    places = 0
    if number >= 1:
        counter = len(str(int(number)))
        places = -(counter + 1)
    else:
        chars = str(number - int(number))[2:]
        for char in chars:
            if char != "0":
                break
            counter += 1
        places = counter

    return places

## Decimal to Sexagesimal Converter

In [19]:
def to_decimal(degrees, arcminutes, arcseconds, is_negative=False):
    decimal_error = 0
    decimal_error_places = 0
        
    if arcminutes < 0 or arcseconds < 0:
        raise ValueError("Arcminutes and arcseconds must be nonnegative")

    if degrees < 0:
        is_negative = True
        degrees *= -1

    if arcseconds != 0:
        decimal_error = get_error(arcseconds) / 60 ** 2
    elif arcminutes != 0:
        decimal_error = get_error(arcminutes) / 60
    else:
        return degrees

    decimal_error_places = get_places(decimal_error)
    full_precision = degrees + arcminutes / 60 + arcseconds / 60 ** 2

    if is_negative:
        full_precision *= -1

    return round(full_precision, decimal_error_places)

Use your Python function to convert these angles to decimal (also, consider how many significant figures you should report!).

a) 11° 54′

b) – 60° 31′ 10′′ (be careful about the sign of the arcminutes and arcseconds when you convert!)

c) – 8° 45′ 15.94′′

In [23]:
to_decimal(11, 54, 0)

11.9

In [24]:
to_decimal(60, 31, 10, is_negative=True)

-60.52

In [25]:
to_decimal(8, 45, 15.94, is_negative=True)

-8.75443

## Sexagesimal to Decimal Converter

In [31]:
def to_sexagesimal(degrees, is_negative=False):
    is_negative = degrees < 0 or is_negative
    degrees = abs(degrees)
    
    sexagesimal_error = get_error(degrees) * 60 ** 2
    sexagesimal_error_places = get_places(sexagesimal_error)
    
    passed_degrees = degrees
    
    degrees = int(passed_degrees)
    arcminutes = int((passed_degrees - degrees) * 60)
    arcseconds = (passed_degrees - degrees - arcminutes / 60) * 60 ** 2

    arcseconds = round(arcseconds, sexagesimal_error_places)
    if sexagesimal_error_places <= 0:
        arcseconds = int(arcseconds)

    if is_negative:
        degrees *= -1
    
    return (degrees, arcminutes, arcseconds)

Use it to covert these angles to degrees, arcminutes, and arcseconds. (Be careful about your significant figures! Maintain the same degree of precision when you change base.)

d) 60.04°

e) 89.99999°

f) – 23.43715°

In [32]:
to_sexagesimal(60.04)

(60, 2, 0)

In [33]:
to_sexagesimal(89.99999)

(89, 59, 60.0)

In [35]:
to_sexagesimal(23.43715, is_negative=True)

(-23, 26, 13.7)

# 4. Determinant of a Matrix

## 2 x 2 Matrix

In [37]:
def det_2x2(matrix):
    a = matrix[0][0]
    b = matrix[0][1]
    c = matrix[1][0]
    d = matrix[1][1]
    return a * d - b * c

## 3 x 3 Matrix

In [38]:
def det_3x3(matrix):
    a = matrix[0][0]
    b = matrix[0][1]
    c = matrix[0][2]
    d = matrix[1][0]
    e = matrix[1][1]
    f = matrix[1][2]
    g = matrix[2][0]
    h = matrix[2][1]
    i = matrix[2][2]

    a_det = det_2x2([[e, f], [h, i]])
    b_det = det_2x2([[d, f], [g, i]])
    c_det = det_2x2([[d, e], [g, h]])
    
    return a * a_det - b * b_det + c * c_det

## N x N Matrix

In [41]:
def det(matrix):

    # make sure matrix is square
    for row in matrix:
        if len(matrix) != len(row):
            raise ValueError("Determinant of non-square matrices is not defined")

    # base case
    if len(matrix) == 2:
        return det_2x2(matrix)
        print "BASE"

    # recursion
    determinate = 0
    for i in range(len(matrix)):
        minor_rows = matrix[1:]
        minor = [[row[j] for j in range(len(row)) if j != i]
                       for row in minor_rows]
        if i % 2 == 0:
            determinate += matrix[0][i] * det(minor)
        else:
            determinate -= matrix[0][i] * det(minor)

    return determinate

In [42]:
det([[1, 2, 3, 5, 8, 2],
     [4, 3, 2, 7, 7, 3],
     [1, 6, 4, 5, 9, 7],
     [3, 7, 2, 9, 5, 7],
     [3, 0, 9, 8, 7, 3],
     [3, 6, 1, 7, 2, 4]])

6796